In [ ]:
import logging
import os
import sys
from dataclasses import dataclass, field
from typing import Optional
import nltk
import numpy as np
from datasets import load_dataset, DatasetDict, Dataset
import evaluate

import transformers
from filelock import FileLock
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    MBart50Tokenizer,
    MBart50TokenizerFast,
    MBartTokenizer,
    MBartTokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version, is_offline_mode
from transformers.utils.versions import require_version

from typing import Tuple
import torch
from torch.utils.data import Dataset
from lion_pytorch import Lion

In [ ]:
try:
    nltk.data.find("tokenizers/punkt")
except (LookupError, OSError):
    if is_offline_mode():
        raise LookupError(
            "Offline mode: run this script without TRANSFORMERS_OFFLINE first to download nltk data files"
        )
    with FileLock(".lock") as lock:
        nltk.download("punkt", quiet=True)

# A list of all multilingual tokenizer which require lang attribute.
MULTILINGUAL_TOKENIZERS = [MBartTokenizer, MBartTokenizerFast, MBart50Tokenizer, MBart50TokenizerFast]

In [ ]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        default="bmd1905/vietnamese-correction",
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where to store the pretrained models downloaded from huggingface.co"},
    )
    use_fast_tokenizer: bool = field(
        default=True,
        metadata={"help": "Whether to use one of the fast tokenizer (backed by the tokenizers library) or not."},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )
    resize_position_embeddings: Optional[bool] = field(
        default=None,
        metadata={
            "help": "Whether to automatically resize the position embeddings if `max_source_length` exceeds "
            "the model's position embeddings."
        },
    )

@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """

    lang: str = field(default="vi", metadata={"help": "Language id for summarization."})

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    text_column: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the column in the datasets containing the full texts (for summarization)."},
    )
    summary_column: Optional[str] = field(
        default=None,
        metadata={"help": "The name of the column in the datasets containing the summaries (for summarization)."},
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a jsonlines or csv file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input evaluation data file to evaluate the metrics (rouge) on "
            "(a jsonlines or csv file)."
        },
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={
            "help": "An optional input test data file to evaluate the metrics (rouge) on " "(a jsonlines or csv file)."
        },
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    max_source_length: Optional[int] = field(
        default=256,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    max_target_length: Optional[int] = field(
        default=256,
        metadata={
            "help": "The maximum total sequence length for target text after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    val_max_target_length: Optional[int] = field(
        default=None,
        metadata={
            "help": "The maximum total sequence length for validation target text after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded. Will default to `max_target_length`."
            "This argument is also used to override the ``max_length`` param of ``model.generate``, which is used "
            "during ``evaluate`` and ``predict``."
        },
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to model maximum sentence length. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
            "efficient on GPU but very bad for TPU."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    max_predict_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of prediction examples to this "
            "value if set."
        },
    )
    num_beams: Optional[int] = field(
        default=None,
        metadata={
            "help": "Number of beams to use for evaluation. This argument will be passed to ``model.generate``, "
            "which is used during ``evaluate`` and ``predict``."
        },
    )
    ignore_pad_token_for_loss: bool = field(
        default=True,
        metadata={
            "help": "Whether to ignore the tokens corresponding to padded labels in the loss computation or not."
        },
    )
    source_prefix: Optional[str] = field(
        default="", metadata={"help": "A prefix to add before every source text (useful for T5 models)."}
    )

    forced_bos_token: Optional[str] = field(
        default=None,
        metadata={
            "help": "The token to force as the first generated token after the decoder_start_token_id."
            "Useful for multilingual models like mBART where the first generated token"
            "needs to be the target language token (Usually it is the target language token)"
        },
    )

    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension in ["csv", "json"], "`train_file` should be a csv or a json file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension in ["csv", "json"], "`validation_file` should be a csv or a json file."
        if self.val_max_target_length is None:
            self.val_max_target_length = self.max_target_length

In [ ]:
model_args = ModelArguments(model_name_or_path='bmd1905/vietnamese-correction')

data_args = DataTrainingArguments(dataset_name='D:/HMI_Lab/vietnamese-correct-als/vi.test.csv',
                                  preprocessing_num_workers=8,
                                  max_source_length=256,
                                  max_target_length=256)
# Cài đặt các siêu tham số huấn luyện
training_args = Seq2SeqTrainingArguments(output_dir="./models/my-vietnamese-correct-als",
                                         learning_rate=1e-5,
                                         per_device_train_batch_size=1,
                                         per_device_eval_batch_size=64,
                                         gradient_accumulation_steps=1,
                                         num_train_epochs=4,
                                         logging_steps=100,
                                         save_steps=500,
                                         overwrite_output_dir=True)

In [ ]:
# Detecting last checkpoint.
last_checkpoint = None
if os.path.isdir(training_args.output_dir) and training_args.do_train and not training_args.overwrite_output_dir:
    last_checkpoint = get_last_checkpoint(training_args.output_dir)
    if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
        raise ValueError(
            f"Output directory ({training_args.output_dir}) already exists and is not empty. "
            "Use --overwrite_output_dir to overcome."
        )
    elif last_checkpoint is not None and training_args.resume_from_checkpoint is None:
        print(
            f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
            "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
        )

# Set seed before initializing model.
set_seed(training_args.seed)

In [ ]:
# Tải mô hình và tokenizer
config = AutoConfig.from_pretrained(
    model_args.config_name if model_args.config_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)
tokenizer = AutoTokenizer.from_pretrained(
    model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path,
    cache_dir=model_args.cache_dir,
    use_fast=model_args.use_fast_tokenizer,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)

config.attention_dropout = 0.2
config.classifier_dropout = 0.4
config.dropout = 0.2

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_args.model_name_or_path,
    from_tf=bool(".ckpt" in model_args.model_name_or_path),
    config=config,
    cache_dir=model_args.cache_dir,
    revision=model_args.model_revision,
    use_auth_token=True if model_args.use_auth_token else None,
)

In [ ]:
model.resize_token_embeddings(len(tokenizer))

if model.config.decoder_start_token_id is None and isinstance(tokenizer, (MBartTokenizer, MBartTokenizerFast)):
    if isinstance(tokenizer, MBartTokenizer):
        model.config.decoder_start_token_id = tokenizer.lang_code_to_id[data_args.lang]
    else:
        model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids(data_args.lang)

if model.config.decoder_start_token_id is None:
    raise ValueError("Make sure that `config.decoder_start_token_id` is correctly defined")

if (
    hasattr(model.config, "max_position_embeddings")
    and model.config.max_position_embeddings < data_args.max_source_length
):
    if model_args.resize_position_embeddings is None:
        print(
            f"Increasing the model's number of position embedding vectors from {model.config.max_position_embeddings} "
            f"to {data_args.max_source_length}."
        )
        model.resize_position_embeddings(data_args.max_source_length)
    elif model_args.resize_position_embeddings:
        model.resize_position_embeddings(data_args.max_source_length)
    else:
        raise ValueError(
            f"`--max_source_length` is set to {data_args.max_source_length}, but the model only has {model.config.max_position_embeddings}"
            f" position encodings. Consider either reducing `--max_source_length` to {model.config.max_position_embeddings} or to automatically "
            "resize the model's position encodings by passing `--resize_position_embeddings`."
        )

prefix = data_args.source_prefix if data_args.source_prefix is not None else ""

In [ ]:
# Chuẩn bị dữ liệu huấn luyện
dataset = load_dataset(
    "csv",
    data_files={"train": data_args.dataset_name},
    cache_dir=model_args.cache_dir,
    use_auth_token=True if model_args.use_auth_token else None,
)

train_dataset = dataset["train"]

# Preprocessing the datasets.
# We need to tokenize inputs and targets.
column_name = train_dataset.column_names

# Get the column names for input/target.
column_names = train_dataset.column_names
if data_args.text_column is None:
    error_column = column_names[0]
else:
    error_column = data_args.text_column

    if error_column not in column_names:
        raise ValueError(
            f"--text_column' value '{data_args.text_column}' needs to be one of: {', '.join(column_names)}"
        )
if data_args.summary_column is None:
    correct_column = column_names[1]
else:
    correct_column = data_args.summary_column
    if correct_column not in column_names:
        raise ValueError(
            f"--summary_column' value '{data_args.summary_column}' needs to be one of: {', '.join(column_names)}"
        )

# Temporarily set max_target_length for training.
max_target_length = data_args.max_target_length
padding = "max_length" if data_args.pad_to_max_length else False

def preprocess_function(examples):
    # remove pairs where at least one record is None

    inputs, targets = [], []
    for i in range(len(examples[error_column])):
        if examples[error_column][i] is not None and examples[correct_column][i] is not None:
            inputs.append(examples[error_column][i])
            targets.append(examples[correct_column][i])

    inputs = [prefix + inp for inp in inputs]
    model_inputs = tokenizer(inputs, max_length=data_args.max_source_length, padding=padding, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length" and data_args.ignore_pad_token_for_loss:
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

if data_args.max_train_samples is not None:
    max_train_samples = min(len(train_dataset), data_args.max_train_samples)
    train_dataset = train_dataset.select(range(max_train_samples))

In [ ]:
# Data collator
label_pad_token_id = -100 if data_args.ignore_pad_token_for_loss else tokenizer.pad_token_id
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8 if training_args.fp16 else None,
)

# Metric
metric = evaluate.load("cer")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    if data_args.ignore_pad_token_for_loss:
        # Replace -100 in the labels as we can't decode them.
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)

    return {"cer": result}

In [ ]:
import pandas as pd
from datasets import Dataset
df = pd.DataFrame(dataset["train"])

# Slice the DataFrame
k = 1
subset_start = len(df) * k // 100
subset_end = len(df) * (k + 10) // 100
subset_df = df.iloc[subset_start:subset_end]

val_ds = Dataset.from_pandas(subset_df)
print(val_ds)
val_tokenized_datasets  = val_ds.map(preprocess_function, batched=True)
print(val_tokenized_datasets[:1])

# Calculate the slices
slices = [(slice(0, k * len(df) // 100), slice((k + 10) * len(df) // 100, None))]

# Apply slicing to generate new DataFrames
cut_dfs = [pd.concat([df.iloc[s1], df.iloc[s2]]) for s1, s2 in slices]
# Concatenate all DataFrames in cut_dfs into a single DataFrame
new_df = pd.concat(cut_dfs)
train_ds = Dataset.from_pandas(new_df)
print(train_ds)
train_tokenized_datasets = train_ds.map(preprocess_function, batched=True)
print(train_tokenized_datasets[:1])


In [ ]:
# Load dataset for the first train
# dataset = load_dataset(
#     "csv",
#     data_files={"train": data_args.dataset_name},
#     cache_dir=model_args.cache_dir,
#     use_auth_token=True if model_args.use_auth_token else None,
#     )

# train_test_dataset = dataset["train"].train_test_split(test_size=0.012)
# test_valid = train_test_dataset['test'].train_test_split(test_size=0.5)

# train_dataset = preprocess_function(train_test_dataset['train'])
# test_dataset = preprocess_function(test_valid['test'])
# val_dataset = preprocess_function(test_valid['train'])

# trainer = Seq2SeqTrainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=val_dataset,
#         tokenizer=tokenizer,
#         data_collator=data_collator,
#         compute_metrics=compute_metrics,
#     )

# Training
# checkpoint = None
# if training_args.resume_from_checkpoint is not None:
#     checkpoint = training_args.resume_from_checkpoint
# elif last_checkpoint is not None:
#     checkpoint = last_checkpoint
# train_result = trainer.train(resume_from_checkpoint=checkpoint)
# trainer.save_model()  # Saves the tokenizer too for easy upload

# metrics = train_result.metrics
# max_train_samples = (
#     data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
# )
# metrics["train_samples"] = min(max_train_samples, len(train_dataset))

# trainer.log_metrics("train", metrics)
# trainer.save_metrics("train", metrics)
# trainer.save_state()

# # Evaluation
# results = {}
# max_length = (
#     training_args.generation_max_length
#     if training_args.generation_max_length is not None
#     else data_args.val_max_target_length
# )
# num_beams = data_args.num_beams if data_args.num_beams is not None else training_args.generation_num_beams
# metrics_val = trainer.evaluate(max_length=max_length, num_beams=num_beams, metric_key_prefix="eval")
# max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(val_dataset)
# metrics["val_samples"] = min(max_eval_samples, len(val_dataset))
# print(metrics_val)

# # Predict
# predict_results = trainer.predict(
#     predict_dataset, metric_key_prefix="predict", max_length=max_length, num_beams=num_beams
# )
# metrics_pre = predict_results.metrics
# max_predict_samples = (
#     data_args.max_predict_samples if data_args.max_predict_samples is not None else len(predict_dataset)
# )
# metrics["predict_samples"] = min(max_predict_samples, len(predict_dataset))
# print(metrics_pre)

In [ ]:
# Initialize our Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=val_tokenized_datasets,
    data_collator=data_collator,
    compute_metrics=compute_metrics,

)

trainer.train()
trainer.save_model()

In [ ]:
trainer.save_model()

In [ ]:
results = {}
max_length = (
    training_args.generation_max_length
    if training_args.generation_max_length is not None
    else data_args.val_max_target_length
)
num_beams = data_args.num_beams if data_args.num_beams is not None else training_args.generation_num_beams
print("*** Evaluate ***")
metrics = trainer.evaluate(max_length=max_length, num_beams=num_beams, metric_key_prefix="eval")
max_eval_samples = data_args.max_eval_samples if data_args.max_eval_samples is not None else len(val_tokenized_datasets)
metrics["eval_samples"] = min(max_eval_samples, len(val_tokenized_datasets))

trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

In [ ]:
predict_results = trainer.predict(val_tokenized_datasets, max_length=max_length, num_beams=num_beams)
metrics = predict_results.metrics
print(metrics)

In [ ]:
from transformers import pipeline

corrector = pipeline("text2text-generation", model="D:\\HMI_Lab\\vietnamese-correct-als\\models\\my-vietnamese-correct-als\\checkpoint-1000", tokenizer=tokenizer)

print(corrector("oke", max_length=256))

In [ ]:
print(corrector("tôdi đau lừng và muốnd đi tiểud", max_length=256))